# Final Setup for Automating DfE Code Reviews

> 3-Component agent

## Tasks


### Schema

In [ ]:
-- Drops

DROP TABLE tasks CASCADE;


-- Creates

CREATE TABLE tasks (
    id SERIAL PRIMARY KEY,
    task_name TEXT NOT NULL,
    task_content TEXT NOT NULL,
    task_instructions TEXT NOT NULL,
    model_answer_1 TEXT NOT NULL,
    model_answer_2 TEXT NOT NULL,
    metadata JSONB
);

CREATE TABLE reviews (
    id SERIAL PRIMARY KEY,
    task_id INT NOT NULL,
    task_name TEXT NOT NULL,
    review_content TEXT NOT NULL,
    submitted_content_1 TEXT NOT NULL,
    submitted_content_2 TEXT NOT NULL,
    metadata JSONB,
    FOREIGN KEY (task_id) REFERENCES tasks (id) ON DELETE CASCADE
);


## Vectorisers


### Task Vectorisers

In [ ]:
SELECT ai.create_vectorizer(   
    'tasks'::regclass,
    formatting => ai.formatting_python_template(
        'Task Title: $task_name\nTask Content: $chunk'
    ),
    embedding => ai.embedding_ollama('nomic-embed-text', 768),
    chunking => ai.chunking_recursive_character_text_splitter('task_content', 
        chunk_size => 512, 
        chunk_overlap => 256
    ),
    destination => 'task_contents_embedding'
);

SELECT ai.create_vectorizer(   
    'tasks'::regclass,
    formatting => ai.formatting_python_template(
        'Task Title: $task_name\nTask Instructions: $chunk'
    ),
    embedding => ai.embedding_ollama('nomic-embed-text', 768),
    chunking => ai.chunking_recursive_character_text_splitter('task_instructions', 
        chunk_size => 512, 
        chunk_overlap => 256
    ),
    destination => 'task_instructions_embedding'
);

SELECT ai.create_vectorizer(   
    'tasks'::regclass,
    formatting => ai.formatting_python_template('$chunk'),
    embedding => ai.embedding_ollama('codellama:13b-instruct', 5120),
    chunking => ai.chunking_recursive_character_text_splitter('model_answer_1'),
    destination => 'task_model_answer_1_embedding'
);

SELECT ai.create_vectorizer(   
    'tasks'::regclass,
    formatting => ai.formatting_python_template('$chunk'),
    embedding => ai.embedding_ollama('codellama:13b-instruct', 5120),
    chunking => ai.chunking_recursive_character_text_splitter('model_answer_2'),
    destination => 'task_model_answer_2_embedding'
);

-- Vectoriser status
SELECT * FROM ai.vectorizer_status;


### Review Vectorisers

In [ ]:
SELECT ai.create_vectorizer(   
    'reviews'::regclass,
    formatting => ai.formatting_python_template('$chunk'),
    embedding => ai.embedding_ollama('nomic-embed-text', 768),
    chunking => ai.chunking_recursive_character_text_splitter('review_content', 
        chunk_size => 512, 
        chunk_overlap => 256
    ),
    destination => 'review_contents_embedding'
);

SELECT ai.create_vectorizer(   
    'reviews'::regclass,
    formatting => ai.formatting_python_template('$chunk'),
    embedding => ai.embedding_ollama('codellama:13b-instruct', 5120),
    chunking => ai.chunking_recursive_character_text_splitter('submitted_content_1'),
    destination => 'submitted_content_1_embedding'
);

SELECT ai.create_vectorizer(   
    'reviews'::regclass,
    formatting => ai.formatting_python_template('$chunk'),
    embedding => ai.embedding_ollama('codellama:13b-instruct', 5120),
    chunking => ai.chunking_recursive_character_text_splitter('submitted_content_2'),
    destination => 'submitted_content_2_embedding'
);

SELECT * FROM ai.vectorizer_status;


In [ ]:
-- Vectoriser execute
SELECT ai.execute_vectorizer(00);

-- Vectoriser status
select * from ai.vectorizer_status;


## RAG Functions

### Basic - General Interaction

In [ ]:
CREATE OR REPLACE FUNCTION general_response(query_text TEXT)
RETURNS TEXT AS $$
DECLARE
    context_chunks TEXT;
    response TEXT;
BEGIN
    WITH relevant_info AS (
    (SELECT 
        'Task Contexts' AS source,
        t1.task_name,
        t1.chunk, t2.chunk
        FROM task_contents_embedding t1
        INNER JOIN task_instructions_embedding t2
        ON t1.task_name = t2.task_name
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', task_title)
        LIMIT 1
        )
        UNION ALL
        (SELECT
        'Model Answer Contexts' AS source,
        t1.task_name,
        t1.chunk, t2.chunk
        FROM task_model_answer_1_embedding t1
        INNER JOIN task_model_answer_2_embedding t2
        ON t1.task_name = t2.task_name
        ORDER BY embedding <=> ai.ollama_embed('codellama:13b-instruct', query_code_1)
        LIMIT 1
        )
        UNION ALL
        (SELECT 
        'Review Context' AS source,
        chunk
        FROM review_contents_embedding
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', task_title)
        LIMIT 1
        )
        UNION ALL
        (SELECT
        'Submission Answer Contexts' AS source,
        t1.task_name,
        t1.chunk, t2.chunk
        FROM submitted_content_1_embedding t1
        INNER JOIN submitted_content_2_embedding t2
        ON t1.task_name = t2.task_name
        ORDER BY embedding <=> ai.ollama_embed('codellama:13b-instruct', query_code_1 || E'\n\n' || query_code_2)
        LIMIT 1
        )
    )
    SELECT string_agg(
        source || E':\n' || chunk, 
        E'\n\n'
    ) 
    INTO context_chunks
    FROM relevant_info;

    SELECT ai.ollama_chat_complete(
        'llama3',
        jsonb_build_array(
        jsonb_build_object('role', 'system', 'content', 'You are a helpful code reviewer. Provide accurate and concise answers based on the given context.'),
        jsonb_build_object(
            'role', 'user',
            'content', query_text || E'\n\nUse the following context to respond:\n' || context_chunks
        )
        )
    )->'message'->>'content' INTO response;

    RETURN response;
END;
$$ LANGUAGE plpgsql;

### Complex - General Interaction

#### V1

In [ ]:
CREATE OR REPLACE FUNCTION general_response(query_text TEXT)
RETURNS TEXT AS $$
DECLARE
    context_chunks TEXT;
    response TEXT;
BEGIN
    WITH relevant_info AS (
        (SELECT 
            'Task Contexts' AS source,
            t1.task_name,
            t1.chunk || E'\n' || t2.chunk AS chunk
            FROM task_contents_embedding t1
            INNER JOIN task_instructions_embedding t2
            ON t1.task_name = t2.task_name
            ORDER BY t1.embedding <=> ai.ollama_embed('nomic-embed-text', query_text)
            LIMIT 1
        )
        UNION ALL
        (SELECT
            'Model Answer Contexts' AS source,
            t1.task_name,
            t1.chunk || E'\n' || t2.chunk AS chunk
            FROM task_model_answer_1_embedding t1
            INNER JOIN task_model_answer_2_embedding t2
            ON t1.task_name = t2.task_name
            ORDER BY t1.embedding <=> ai.ollama_embed('codellama:13b-instruct', query_text)
        )
        UNION ALL
        (SELECT 
            'Review Context' AS source,
            '' AS task_name,
            chunk
            FROM review_contents_embedding
            ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', query_text)
        )
        UNION ALL
        (SELECT
            'Submission Answer Contexts' AS source,
            t1.task_name,
            t1.chunk || E'\n' || t2.chunk AS chunk
            FROM submitted_content_1_embedding t1
            INNER JOIN submitted_content_2_embedding t2
            ON t1.task_name = t2.task_name
            ORDER BY t1.embedding <=> ai.ollama_embed('codellama:13b-instruct', query_text)
        )
    )
    SELECT string_agg(
        source || E': ' || task_name || E'\n' || chunk, 
        E'\n\n'
    ) 
    INTO context_chunks
    FROM relevant_info;

    SELECT ai.ollama_chat_complete(
        'llama3',
        jsonb_build_array(
            jsonb_build_object('role', 'system', 'content', 'You are a helpful code reviewer. Provide accurate and concise answers based on the given context.'),
            jsonb_build_object(
                'role', 'user',
                'content', query_text || E'\n\nUse the following context to respond:\n' || context_chunks
            )
        )
    )->'message'->>'content' INTO response;

    RETURN response;
END;
$$ LANGUAGE plpgsql;


-- Test query for general_response function
SELECT general_response(
    'Provide a summary of all tasks and their objectives as outlined in the task instructions. 
    Additionally, identify any associated reviews and summarize their content. 
    Ensure the summary is structured as follows:
    
    - Task Name: [Task Name]
      Objective: [Extracted from Task Instructions]
      Reviews:
        1. [First Review Summary]
        2. [Second Review Summary]
    
    Include any model answers for each task as supplementary information.'
);


#### V2

In [ ]:
CREATE OR REPLACE FUNCTION general_response(query_text TEXT)
RETURNS TEXT AS $$
DECLARE
    context_chunks TEXT;
    response TEXT;
BEGIN
    WITH relevant_info AS (
        -- Task Context
        (SELECT 
            'Task Contexts' AS source,
            t1.task_name,
            t1.chunk || E'\n' || t2.chunk AS chunk
        FROM task_contents_embedding t1
        INNER JOIN task_instructions_embedding t2
        ON t1.task_name = t2.task_name
        ORDER BY t1.embedding <=> ai.ollama_embed('nomic-embed-text', query_text)
        )

        UNION ALL

        -- Model Answer Context
        (SELECT
            'Model Answer Contexts' AS source,
            t1.task_name,
            t1.chunk || E'\n' || t2.chunk AS chunk
        FROM task_model_answer_1_embedding t1
        INNER JOIN task_model_answer_2_embedding t2
        ON t1.task_name = t2.task_name
        ORDER BY t1.embedding <=> ai.ollama_embed('codellama:13b-instruct', query_text)
        )

        UNION ALL

        -- Review Context
        (SELECT 
            'Review Context' AS source,
            '' AS task_name,
            chunk
        FROM review_contents_embedding
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', query_text)
        )

        UNION ALL

        -- Submission Answer Context
        (SELECT
            'Submission Answer Contexts' AS source,
            t1.task_name,
            t1.chunk || E'\n' || t2.chunk AS chunk
        FROM submitted_content_1_embedding t1
        INNER JOIN submitted_content_2_embedding t2
        ON t1.task_name = t2.task_name
        ORDER BY t1.embedding <=> ai.ollama_embed('codellama:13b-instruct', query_text)
        )
    )
    SELECT string_agg(
        source || E': ' || COALESCE(task_name, 'N/A') || E'\n' || chunk,
        E'\n\n'
    ) 
    INTO context_chunks
    FROM relevant_info;

    -- Construct the final response using AI model
    SELECT ai.ollama_chat_complete(
        'llama3',
        jsonb_build_array(
            jsonb_build_object(
                'role', 'system', 
                'content', 'You are a helpful assistant providing structured responses based on the given context.'
            ),
            jsonb_build_object(
                'role', 'user',
                'content', query_text || E'\n\nUse the following context to respond:\n' || COALESCE(context_chunks, 'No relevant context found.')
            )
        )
    )->'message'->>'content' 
    INTO response;

    RETURN response;
END;
$$ LANGUAGE plpgsql;


-- Test query for general_response function
SELECT general_response(
    'Provide a summary of all tasks and their objectives as outlined in the task instructions. 
    Additionally, identify any associated reviews and summarize their content. 
    Ensure the summary is structured as follows:
    
    - Task Name: [Task Name]
      Objective: [Extracted from Task Instructions]
      Reviews:
        1. [First Review Summary]
        2. [Second Review Summary]
    
    Include any model answers for each task as supplementary information.'
);


#### V4

In [ ]:
CREATE OR REPLACE FUNCTION general_response(query_text TEXT)
RETURNS TEXT AS $$
DECLARE
    context_chunks TEXT;
    response TEXT;
BEGIN
    WITH relevant_info AS (
        -- Task Contexts with clear task separation
        (SELECT 
            t1.task_name,
            '===== Task: ' || t1.task_name || ' =====' || E'\n'
            || 'Task Context:\n' || t1.chunk || E'\n'
            || 'Instructions:\n' || t2.chunk AS chunk
        FROM task_contents_embedding t1
        INNER JOIN task_instructions_embedding t2
        ON t1.task_name = t2.task_name
        ORDER BY t1.embedding <=> ai.ollama_embed('nomic-embed-text', query_text)
        LIMIT 3)

        UNION ALL

        -- Model Answer Contexts
        (SELECT
            t1.task_name,
            '===== Task: ' || t1.task_name || ' =====' || E'\n'
            || 'Model Answer 1:\n' || t1.chunk || E'\n'
            || 'Model Answer 2:\n' || t2.chunk AS chunk
        FROM task_model_answer_1_embedding t1
        INNER JOIN task_model_answer_2_embedding t2
        ON t1.task_name = t2.task_name
        ORDER BY t1.embedding <=> ai.ollama_embed('codellama:13b-instruct', query_text)
        LIMIT 2)

        UNION ALL

        -- Review Contexts
        (SELECT 
            '' AS task_name,
            '===== Review Context =====' || E'\n' || chunk AS chunk
        FROM review_contents_embedding
        ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', query_text)
        LIMIT 5)

        UNION ALL

        -- Submission Answer Contexts
        (SELECT
            t1.task_name,
            '===== Task: ' || t1.task_name || ' =====' || E'\n'
            || 'Submission Answer 1:\n' || t1.chunk || E'\n'
            || 'Submission Answer 2:\n' || t2.chunk AS chunk
        FROM submitted_content_1_embedding t1
        INNER JOIN submitted_content_2_embedding t2
        ON t1.task_name = t2.task_name
        ORDER BY t1.embedding <=> ai.ollama_embed('codellama:13b-instruct', query_text)
        LIMIT 2)
    )
    SELECT string_agg(chunk, E'\n\n') 
    INTO context_chunks
    FROM relevant_info;

    -- Generate the response using AI model
    SELECT ai.ollama_chat_complete(
        'llama3',
        jsonb_build_array(
            jsonb_build_object(
                'role', 'system',
                'content', 'You are a helpful assistant. Answer the query using the structured context provided, clearly separated by task.'
            ),
            jsonb_build_object(
                'role', 'user',
                'content', query_text || E'\n\nStructured Context:\n' || COALESCE(context_chunks, 'No relevant context found.')
            )
        )
    )->'message'->>'content' 
    INTO response;

    RETURN response;
END;
$$ LANGUAGE plpgsql;

-- Test query for general_response function
SELECT general_response(
    'Provide a summary of all tasks and their objectives as outlined in the task instructions. 
    Additionally, identify any associated reviews and summarize their content. 
    Ensure the summary is structured as follows:
    
    - Task Name: [Task Name]
      Objective: [Extracted from Task Instructions]
      Reviews:
        1. [First Review Summary]
        2. [Second Review Summary]
    
    Include any model answers for each task as supplementary information.'
);

SELECT general_response(
    'Summarize the objectives of each task from the instructions, and provide any associated reviews. 
    For each task, also list model answers and submissions as available. Ensure the tasks are clearly separated in your response.'
);



### Own

In [ ]:
CREATE OR REPLACE FUNCTION generate_review(query_code_1 TEXT, query_code_2 TEXT, task_title TEXT, file_name TEXT)
RETURNS TEXT AS $$
DECLARE
  context_chunks TEXT;
  response TEXT;
BEGIN
  WITH relevant_info AS (
    (SELECT 
      'Task Contexts' AS source,
      t1.task_name,
      t1.chunk, t2.chunk
      FROM task_contents_embedding t1
      INNER JOIN task_instructions_embedding t2
      ON t1.task_name = t2.task_name
      ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', task_title)
      LIMIT 1
    )
    UNION ALL
    (SELECT
      'Model Answer Contexts' AS source,
      t1.task_name,
      t1.chunk, t2.chunk
      FROM task_model_answer_1_embedding t1
      INNER JOIN task_model_answer_2_embedding t2
      ON t1.task_name = t2.task_name
      ORDER BY embedding <=> ai.ollama_embed('codellama:13b-instruct', query_code_1)
      LIMIT 1
    )
    UNION ALL
    (SELECT 
      'Review Context' AS source,
      chunk
      FROM review_contents_embedding
      ORDER BY embedding <=> ai.ollama_embed('nomic-embed-text', task_title)
      LIMIT 1
    )
    UNION ALL
    (SELECT
      'Submission Answer Contexts' AS source,
      t1.task_name,
      t1.chunk, t2.chunk
      FROM submitted_content_1_embedding t1
      INNER JOIN submitted_content_2_embedding t2
      ON t1.task_name = t2.task_name
      ORDER BY embedding <=> ai.ollama_embed('codellama:13b-instruct', query_code_1 || E'\n\n' || query_code_2)
      LIMIT 1
    )
  )
  SELECT string_agg(
    source || E':\n' || chunk, 
    E'\n\n'
  ) 
  INTO context_chunks
  FROM relevant_info;

  SELECT ai.ollama_chat_complete(
    'llama3',
    jsonb_build_array(
      jsonb_build_object(
        'role', 'system', 
        'content', 'You are an expert code reviewer. Your task is to review the given code (query_text) and ensure it matches the task instructions. Provide a detailed review, highlighting strengths, areas for improvement, and any discrepancies between the code and the task requirements.'
      ),
      jsonb_build_object(
        'role', 'user',
        'content', 'Task Name: ' || task_title || E'\n\n' ||
        'File name:\n' || file_name || E'\n\n' ||
        'Code to review:\n' || query_text_1 || query_text_2 || E'\n\n' ||
        'Use the following context for your review:\n' || context_chunks || E'\n\n' ||
        'Please provide a comprehensive code review, focusing on how well the code matches the task instructions. Include specific suggestions for improvement if necessary.'
      )
    )
  )->'message'->>'content' INTO response;

  RETURN response;
END;
$$ LANGUAGE plpgsql;